In [1]:
import pandas as pd
from random import randint
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt

In [2]:
meters = pd.read_excel('data/gorilla_test_data.xlsx', sheet_name = 'meter_list')
forecast = pd.read_excel('data/gorilla_test_data.xlsx', sheet_name = 'forecast_table')
rate = pd.read_excel('data/gorilla_test_data.xlsx', sheet_name = 'rate_table')

In [3]:
meters

,meter_id,aq_kwh,exit_zone
0,14676236,28978,EA1
1,34509937,78324,SO1
2,50264822,265667,NT1
3,88357331,484399,SE2


In [4]:
# forecast.info()
# forecast.head()

In [5]:
# rate.info()
# rate.head()

In [6]:
rate["aq_max_kwh"] = rate["aq_max_kwh"].replace(np.nan,np.inf)

In [8]:
rate.head()

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
1,2020-04-01,EA1,73200,732000.0,0.1980
2,2020-04-01,EA1,732000,inf,0.2875
3,2020-04-01,EA2,0,73200.0,0.2970
4,2020-04-01,EA2,73200,732000.0,0.1524


In [9]:
# keep only data corresponding to the exit zone we have in meter_list
rate = rate[rate['exit_zone'].isin(list(meters['exit_zone']))]
# merge the rate and meter dataframes to have the meter_id with the rates
meters_and_rate = pd.merge(rate, meters, on="exit_zone")

In [11]:
# keep only rows with the correct subset of rates
meters_and_rate = meters_and_rate[(meters_and_rate['aq_kwh'] >= meters_and_rate['aq_min_kwh']) & (meters_and_rate['aq_kwh'] < meters_and_rate['aq_max_kwh'])]

In [11]:
# check meters in forecast corresponds to meter_list
forecast['meter_id'].unique() == meters['meter_id']

0    True
1    True
2    True
3    True
Name: meter_id, dtype: bool

In [12]:
# merge with forecast
meters_and_rate_with_forecast = pd.merge(meters_and_rate, forecast, on="meter_id", suffixes=('_rate', '_forecast'))
# keep rows with the correct dates only
meters_and_rate_with_forecast = meters_and_rate_with_forecast[meters_and_rate_with_forecast['date_forecast'] >= meters_and_rate_with_forecast['date_rate']]
meters_and_rate_with_forecast = meters_and_rate_with_forecast.loc[meters_and_rate_with_forecast.groupby(['meter_id', 'date_forecast']).date_rate.idxmax()]

In [12]:

meters_and_rate_with_forecast

,date_rate,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,meter_id,aq_kwh,date_forecast,kwh
0,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978,2020-06-01,22.070768
1,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978,2020-06-02,19.170720
2,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978,2020-06-03,23.555111
3,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978,2020-06-04,18.220712
4,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978,2020-06-05,14.196134
...,...,...,...,...,...,...,...,...,...
21320,2022-04-01,SE2,73200,732000.0,0.3647,88357331,484399,2022-09-27,441.014725
21321,2022-04-01,SE2,73200,732000.0,0.3647,88357331,484399,2022-09-28,441.512055
21322,2022-04-01,SE2,73200,732000.0,0.3647,88357331,484399,2022-09-29,437.240657
21323,2022-04-01,SE2,73200,732000.0,0.3647,88357331,484399,2022-09-30,517.608354


In [15]:
# compute the cost for each daily consumption forecast
meters_and_rate_with_forecast['cost'] = round(meters_and_rate_with_forecast['rate_p_per_kwh']*meters_and_rate_with_forecast['kwh']*0.01,2)
meters_and_rate_with_forecast.head()

,date_rate,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,meter_id,aq_kwh,date_forecast,kwh,cost
0,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978,2020-06-01,22.070768,0.06
1,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978,2020-06-02,19.170720,0.05
2,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978,2020-06-03,23.555111,0.06
3,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978,2020-06-04,18.220712,0.05
4,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978,2020-06-05,14.196134,0.04


In [18]:
# compute the total consumption per meter
total_estimated_consumption = meters_and_rate_with_forecast.groupby('meter_id')['kwh'].sum()
total_estimated_consumption

meter_id
14676236     28978.0
34509937     78324.0
50264822    265667.0
88357331    484399.0
Name: kwh, dtype: float64

In [19]:
# compute the total consumption per meter
total_cost = meters_and_rate_with_forecast.groupby('meter_id')['cost'].sum()
total_cost

meter_id
14676236     100.20
34509937     275.68
50264822     731.32
88357331    1433.14
Name: cost, dtype: float64

In [20]:
results = pd.merge(total_estimated_consumption, total_cost, on="meter_id").reset_index()
results = results.rename(columns={'kwh': 'total_estimated_consumption', 'cost': 'total_cost'})
results

,meter_id,total_estimated_consumption,total_cost
0,14676236,28978.0,100.20
1,34509937,78324.0,275.68
2,50264822,265667.0,731.32
3,88357331,484399.0,1433.14


Question 2

In [22]:
exit_zone = rate['exit_zone'].unique()

def meter_generator(list_size):
    meter_dict = {}
    meter_ids = np.random.randint(10**(7), (10**8)-1, size=list_size) # random int with 8 digits
    aqs = np.random.randint(0,10**6, size=list_size) # random AQ, range is arbitrary
    exit_zones = np.random.choice(exit_zone, size=list_size) # random zone from the existing ones
    meter_dict['meter_id'] = meter_ids
    meter_dict['exit_zone'] = exit_zones
    meter_dict['aq_kwh'] = aqs
    return pd.DataFrame(meter_dict)
    

In [25]:
meter_df_test = meter_generator(5)
print(len(meter_df_test['meter_id']))
print(len(meter_df_test['exit_zone']))
print(len(meter_df_test['aq_kwh']))
meter_df_test

5
5
5


,meter_id,exit_zone,aq_kwh
0,95273614,NT1,11075
1,95417483,SE2,568932
2,16088701,EA1,622189
3,77861687,SE2,861720
4,14540699,EA1,538622


Question 3

In [27]:
def forecast_generator(meters, start_date, duration):
    consumption_dict = {}
    n_meters = len(meters['meter_id'])
    date_list = [start_date + datetime.timedelta(days=n) for n in range(duration)]
    n_days = len(date_list)
    kwh = np.random.uniform(10,900, size=n_meters*n_days)
    consumption_dict['meter_id'] = np.repeat(meters['meter_id'], n_days)
    consumption_dict['date'] = date_list*n_meters
    consumption_dict['kwh'] = kwh
    return pd.DataFrame(consumption_dict)

In [30]:
start_date = datetime.date(2023, 8, 1)
duration = 3
consumption = forecast_generator(meter_df_test, start_date, duration)
print(consumption.info())
consumption.head()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 0 to 4
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   meter_id  15 non-null     int32  
 1   date      15 non-null     object 
 2   kwh       15 non-null     float64
dtypes: float64(1), int32(1), object(1)
memory usage: 420.0+ bytes
None


,meter_id,date,kwh
0,95273614,2023-08-01,765.465149
0,95273614,2023-08-02,550.072262
0,95273614,2023-08-03,898.353655
1,95417483,2023-08-01,722.013952
1,95417483,2023-08-02,151.370185


Question 4

In [32]:
def get_transportation_cost(meter_list, forecast, rate):
    meters_and_rate = pd.merge(rate, meter_list, on="exit_zone")
    meters_and_rate = meters_and_rate[(meters_and_rate['aq_kwh'] >= meters_and_rate['aq_min_kwh']) & (meters_and_rate['aq_kwh'] < meters_and_rate['aq_max_kwh'])]
    meters_and_rate_with_forecast = pd.merge(meters_and_rate, forecast, on="meter_id", suffixes=('_rate', '_forecast'))
    meters_and_rate_with_forecast = meters_and_rate_with_forecast[meters_and_rate_with_forecast['date_forecast'] >= meters_and_rate_with_forecast['date_rate']]
    meters_and_rate_with_forecast=meters_and_rate_with_forecast.loc[meters_and_rate_with_forecast.groupby(['meter_id', 'date_forecast']).date_rate.idxmax()]
    meters_and_rate_with_forecast['cost'] = round(meters_and_rate_with_forecast['rate_p_per_kwh']*meters_and_rate_with_forecast['kwh']*0.01,2)
    total_estimated_consumption = round(meters_and_rate_with_forecast.groupby('meter_id')['kwh'].sum(),0)
    total_cost = meters_and_rate_with_forecast.groupby('meter_id')['cost'].sum()
    transportation_cost = pd.merge(total_estimated_consumption, total_cost, on="meter_id").reset_index()
    transportation_cost = transportation_cost.rename(columns={'kwh': 'total_estimated_consumption', 'cost': 'total_cost'})
    return transportation_cost

In [33]:
# to benchmark with different meter list sizes
list_sizes = [5, 50, 500, 5000, 50000]
meter_lists = []
for size in list_sizes:
    meter_lists.append(meter_generator(size))

start_date = datetime.date(2021, 1, 1)

# to benchmark with different period lengths 
n_days = [5, 365, 1000]

for meter in meter_lists:
    for n in n_days:
        start_time = time.time()
        forecast = forecast_generator(meter, start_date, n)
        transportation_cost = get_transportation_cost(meter, forecast, rate)
        end_time = time.time()
        print(f"Computation of transportation cost for {len(meter)} meters with {n} days forcast.")
        print("Time passed", end_time-start_time)
        print()

Computation of transportation cost for 5 meters with 5 days forcast.
Time passed 0.04238247871398926

Computation of transportation cost for 5 meters with 365 days forcast.
Time passed 0.5914888381958008

Computation of transportation cost for 5 meters with 1000 days forcast.
Time passed 0.988224983215332

Computation of transportation cost for 50 meters with 5 days forcast.
Time passed 0.1218724250793457

Computation of transportation cost for 50 meters with 365 days forcast.
Time passed 3.836216688156128

Computation of transportation cost for 50 meters with 1000 days forcast.
Time passed 13.333918809890747

Computation of transportation cost for 500 meters with 5 days forcast.
Time passed 0.749840497970581

Computation of transportation cost for 500 meters with 365 days forcast.
Time passed 44.17945408821106

Computation of transportation cost for 500 meters with 1000 days forcast.
Time passed 110.31116914749146

Computation of transportation cost for 5000 meters with 5 days forcast

MemoryError: Unable to allocate 3.73 GiB for an array with shape (1, 500320000) and data type int64

This function is not adapted for larger sets of data. Here with a meter_list of 50000 meters and a forecast period of 1000 days, I got the following error:
MemoryError: Unable to allocate 3.73 GiB for an array with shape (1, 500320000) and data type int64.

Question 5

We can see that for larger sets of data, it can take quite some time, for example it took about 40 minutes to compute the cost with 50000 meters and 365 days forecast.
To have a faster process and to solve the out of memory issue, we could use parallel computing. If we have a big amount of meters, we could split by meters and use the code as it is. If we have long forecast period, we can split by forecast but we should make sure to keep forecast for the same meter in the same split.
In python, it seems we can use the Dask library, that processes large tabular data by parallelizing pandas.

In the cost calculation, the step to get the rate_p_per_kwh depending on the forecast may be improved. I first merge different dataframes giving me something with too much data and then filter it, which does not seem optimal.